In [1]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

In [2]:
# Define the URL of the page with the images
url = "https://www.wmagazine.com/fashion/milan-fashion-week-spring-2024-runway-photos"

# Send a GET request to the URL
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    image_elements = soup.find_all('figure', class_='W0r')

    save_directory = 'scraped_images'
    os.makedirs(save_directory, exist_ok=True)

    for index, image_element in enumerate(image_elements):
        image_url = image_element.find('img')['src']

        image_response = requests.get(image_url)

        if image_response.status_code == 200:
            image_data = BytesIO(image_response.content)

            image = Image.open(image_data)

            image_filename = os.path.join(save_directory, f'image_{index}.jpg')

            image.save(image_filename)

            print(f"Image {index} saved as {image_filename}")
        else:
            print(f"Failed to retrieve image {index}")
else:
    
    print("Failed to retrieve the page.")

Image 0 saved as scraped_images\image_0.jpg
Image 1 saved as scraped_images\image_1.jpg
Image 2 saved as scraped_images\image_2.jpg
Image 3 saved as scraped_images\image_3.jpg
Image 4 saved as scraped_images\image_4.jpg
Image 5 saved as scraped_images\image_5.jpg
Image 6 saved as scraped_images\image_6.jpg
Image 7 saved as scraped_images\image_7.jpg
Image 8 saved as scraped_images\image_8.jpg
Image 9 saved as scraped_images\image_9.jpg
Image 10 saved as scraped_images\image_10.jpg
Image 11 saved as scraped_images\image_11.jpg
Image 12 saved as scraped_images\image_12.jpg
Image 13 saved as scraped_images\image_13.jpg
Image 14 saved as scraped_images\image_14.jpg
Image 15 saved as scraped_images\image_15.jpg
Image 16 saved as scraped_images\image_16.jpg
Image 17 saved as scraped_images\image_17.jpg
Image 18 saved as scraped_images\image_18.jpg
Image 19 saved as scraped_images\image_19.jpg
Image 20 saved as scraped_images\image_20.jpg
Image 21 saved as scraped_images\image_21.jpg
Image 22

In [3]:
# import cv2
# import numpy as np
# import webcolors
# #from colormap import rgb2hex, hex2rgb, closest_color_name

In [4]:
# def rgb_to_color_name(rgb):
#     # Define color ranges and corresponding color names
#     color_mapping = {
#         (255, 0, 0): 'Red',
#         (0, 255, 0): 'Green',
#         (0, 0, 255): 'Blue',
#         # Add more color mappings as needed
#     }
    
#     # Check the closest match in the mapping
#     closest_match = min(color_mapping, key=lambda color: np.linalg.norm(np.array(color) - np.array(rgb))
#     return color_mapping.get(closest_match, 'Unknown')

In [5]:
# # Load the image
# image = cv2.imread('scraped_images\image_0.jpg')  # Replace 'image_path.jpg' with the path to your image

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a mask for the model
# _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

# # Invert the mask (black background, white model)
# mask = cv2.bitwise_not(mask)

# # Bitwise-AND the original image with the mask to keep only the model
# result = cv2.bitwise_and(image, image, mask=mask)

# # Convert the result to RGB format
# result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

# # Calculate the histogram of colors in the model region
# hist = cv2.calcHist([result_rgb], [0, 1, 2], mask, [256, 256, 256], [0, 256, 0, 256, 0, 256])

# # Find the most dominant color (the peak in the histogram)
# dominant_color = np.unravel_index(hist.argmax(), hist.shape)

# # Convert the BGR values to RGB
# main_color = (int(dominant_color[2]), int(dominant_color[1]), int(dominant_color[0]))
# # Convert the RGB values to color names
# # Convert the RGB values to color names
# color_name = rgb_to_color_name(main_color)

# print("Main Color (RGB):", main_color)
# print("Main color in basic palette:", color_name)

In [1]:
import cv2
import numpy as np
import os
import csv

# Function to classify an RGB color to the nearest base color
def classify_color(rgb_color, base_colors):
    # Find the nearest base color by calculating the minimum Euclidean distance
    min_distance = float('inf')
    nearest_base_color = "Unknown"
    
    for color_name, color_values in base_colors.items():
        for shade_name, base_color in color_values.items():
            distance = np.linalg.norm(np.array(rgb_color) - np.array(base_color))
            if distance < min_distance:
                min_distance = distance
                nearest_base_color = color_name
    
    return nearest_base_color

# Function to classify an RGB color to its color name based on a dataset
def classify_to_color_name(rgb_color, color_dataset):
    min_distance = float('inf')
    nearest_color_name = "Unknown"
    for color_name, dataset_rgb in color_dataset.items():
        dataset_rgb_values = [int(value) for value in dataset_rgb.split(';')]
        distance = np.linalg.norm(np.array(rgb_color) - np.array(dataset_rgb_values))
        if distance < min_distance:
            min_distance = distance
            nearest_color_name = color_name
    return nearest_color_name

# Define the folder containing the images
image_folder = 'scraped_images'

# Get a list of all image files in the folder
image_files = [file for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

base_colors = {
    'Red': {
        'Red': (255, 0, 0),
        'Bright Red': (238, 75, 43),
        'Cadmium Red': (210, 43, 43),
        'Cardinal Red': (196, 30, 58),
        'Carmine (Red)': (215, 0, 64),
        'Cherry (Red)': (210, 4, 45),
        'Dark Red': (139, 0, 0),
        'Red Orange': (255, 68, 51),
        'Scarlet': (255, 36, 0),
        'Brick Red': (170, 74, 68),
        'Blood Red': (255, 87, 51),
        'Burgundy': (128, 0, 32),
    },
    'Green': {
        'Green': (0, 255, 0),
        'Army Green': (69, 75, 27),
        'Aqua': (0, 255, 255),
        'Baby Blue': (173, 216, 230),
        'Emerald': (0, 204, 102),
        'Forest Green': (34, 139, 34),
        'Lime Green': (50, 205, 50),
        'Olive Drab': (107, 142, 35),
        'Mint Green': (152, 255, 152),
        'Teal': (0, 128, 128),
        'Dark Green': (40,67,34),
    },
    'Blue': {
        'Blue': (0, 0, 255),
        'Bright Blue': (0, 150, 255),
        'Dark Blue': (0, 0, 139),
        'Egyptian Blue': (16, 52, 166),
        'Navy Blue': (0, 0, 128),
        'Royal Blue': (65, 105, 225),
        'Powder Blue': (176, 224, 230),
        'Sapphire': (15, 82, 186),
        'Turquoise': (64, 224, 208),
        'Cerulean': (0, 123, 167),
        'Light Grayish Blue':(192,204,221),
    },
    'Yellow': {
        'Yellow': (255, 255, 0),
        'Bright Yellow': (255, 234, 0),
        'Cadmium Yellow': (255, 246, 0),
        'Canary Yellow': (255, 255, 102),
        'Golden Yellow': (255, 223, 0),
        'Lemon Yellow': (255, 244, 79),
        'Mustard Yellow': (255, 219, 88),
        'Pineapple': (86, 60, 13),
        'Banana': (255, 209, 42),
        'Daffodil': (255, 204, 51),
    },
    'Gray': {
        'Gray': (128, 128, 128),
        'Dark Gray': (169, 169, 169),
        'Light Gray': (211, 211, 211),
        'Charcoal': (54, 69, 79),
        'Slate Gray': (112, 128, 144),
        'Steel Gray': (96, 96, 96),
        'Cloudy Gray': (169, 169, 169),
        'Silver': (192, 192, 192),
        'Smoke': (115, 130, 118),
        'Pewter': (168, 168, 168),
    },
    'Purple': {
        'Purple': (128, 0, 128),
        'Lavender': (230, 230, 250),
        'Violet': (148, 0, 211),
        'Indigo': (75, 0, 130),
        'Mauve': (224, 176, 255),
        'Plum': (221, 160, 221),
        'Eggplant': (97, 64, 81),
        'Lilac': (200, 162, 200),
        'Amethyst': (153, 102, 204),
        'Orchid': (218, 112, 214),
    },
    'Orange': {
        'Orange': (255, 165, 0),
        'Pumpkin': (255, 117, 24),
        'Tangerine': (255, 166, 77),
        'Apricot': (251, 206, 177),
        'Sunset': (253, 94, 83),
        'Cantaloupe': (255, 240, 153),
        'Peach': (255, 229, 180),
        'Terracotta': (204, 78, 92),
        'Burnt Orange': (204, 85, 0),
        'Rust': (183, 65, 14),
    },
    'Pink': {
        'Pink': (255, 192, 203),
        'Bubblegum Pink': (255, 105, 180),
        'Carnation Pink': (255, 182, 193),
        'Magenta Pink': (204, 51, 139),
        'Rose': (255, 0, 127),
        'Watermelon': (240, 92, 133),
        'Hot Pink': (255, 105, 180),
        'Fuchsia': (255, 0, 255),
        'Lavender Pink': (251, 174, 210),
        'Salmon': (250, 128, 114),
    },
    'Lime': {
        'Lime': (50, 205, 50),
        'Chartreuse': (127, 255, 0),
        'Lawn Green': (124, 252, 0),
        'Yellow Green': (154, 205, 50),
        'Green Yellow': (173, 255, 47),
        'Spring Green': (0, 255, 127),
        'Apple Green': (141, 182, 0),
        'Kelly Green': (76, 187, 23),
        'Mint': (189, 252, 201),
        'Neon Green': (57, 255, 20),
    },
    'Cyan': {
        'Cyan': (0, 255, 255),
        'Aqua': (0, 255, 255),
        'Teal': (0, 128, 128),
        'Turquoise': (64, 224, 208),
        'Electric Blue': (125, 249, 255),
        'Robin Egg Blue': (0, 204, 204),
        'Light Sea Green': (32, 178, 170),
        'Sky Blue': (135, 206, 235),
        'Ice Blue': (173, 216, 230),
        'Aquamarine': (127, 255, 212),
    },
    'Magenta': {
        'Magenta': (255, 0, 255),
        'Fuchsia': (255, 0, 255),
        'Orchid': (218, 112, 214),
        'Deep Pink': (255, 20, 147),
        'Hot Magenta': (255, 105, 180),
        'Raspberry': (135, 38, 87),
        'Mauve': (224, 176, 255),
        'Berry': (153, 50, 204),
        'Rose': (255, 0, 127),
        'Lavender Blush': (255, 240, 245),
    },
    'Black': {
        'Black': (0, 0, 0),
        'Charcoal': (54, 69, 79),
        'Ebony': (85, 93, 80),
        'Midnight': (25, 25, 112),
        'Onyx': (53, 56, 57),
        'Jet Black': (0, 0, 0),
        'Slate Black': (46, 50, 60),
        'Pitch Black': (0, 0, 0),
        'Raven': (25, 14, 21),
        'Obsidian': (20, 29, 38),
    },
    'White': {
        'White': (255, 255, 255),
        'Snow White': (255, 250, 250),
        'Ivory': (255, 255, 240),
        'Pearl': (240, 234, 214),
        'Cream': (255, 253, 208),
        'Linen': (250, 240, 230),
        'Alabaster': (240, 248, 255),
        'Bone': (219, 217, 209),
        'Eggshell': (252, 230, 201),
        'Vanilla': (243, 229, 171),
        'Light grayish violet':(231,230,233),
    },
    'Olive': {
        'Olive': (128, 128, 0),
        'Khaki': (240, 230, 140),
        'Camouflage Green': (143, 188, 143),
        'Olive Drab': (107, 142, 35),
        'Army Green': (75, 83, 32),
        'Avocado': (86, 130, 3),
        'Forest Green': (34, 139, 34),
        'Moss Green': (173, 223, 173),
        'Loden Green': (130, 138, 88),
        'Camo': (117, 133, 120),
    },
    'Maroon': {
        'Maroon': (128, 0, 0),
        'Burgundy': (128, 0, 32),
        'Crimson': (220, 20, 60),
        'Bordeaux': (123, 23, 48),
        'Ruby': (224, 17, 95),
        'Garnet': (115, 54, 53),
        'Wine': (114, 47, 55),
        'Merlot': (115, 54, 66),
        'Sangria': (146, 0, 10),
        'Mahogany': (75, 0, 13),
    },
    'Teal': {
        'Teal': (0, 128, 128),
        'Turquoise': (64, 224, 208),
        'Aquamarine': (127, 255, 212),
        'Cyan': (0, 255, 255),
        'Robin Egg Blue': (0, 204, 204),
        'Jade': (0, 168, 107),
        'Tiffany Blue': (10, 186, 181),
        'Blue-Green': (13, 152, 186),
        'Emerald': (0, 204, 153),
        'Mint': (152, 255, 152),
    },
    'Silver': {
        'Silver': (192, 192, 192),
        'Platinum': (214, 214, 208),
        'Nickel': (175, 175, 175),
        'Pewter': (168, 168, 168),
        'Tin': (171, 171, 171),
        'Titanium': (194, 194, 194),
        'Steel': (179, 179, 179),
        'Ash': (192, 192, 192),
        'Mica': (189, 184, 179),
        'Quicksilver': (166, 166, 166),
    },
    'Gold': {
        'Gold': (255, 215, 0),
        'Goldenrod': (218, 165, 32),
        'Brass': (181, 166, 66),
        'Amber': (255, 191, 0),
        'Honey': (255, 183, 51),
        'Copper': (184, 115, 51),
        'Bronze': (205, 127, 50),
        'Sunset Gold': (252, 202, 3),
        'Topaz': (255, 200, 124),
        'Saffron': (244, 196, 48),
    },
    'Brown': {
        'Brown': (165, 42, 42),
        'Chocolate': (210, 105, 30),
        'Sienna': (160, 82, 45),
        'Auburn': (165, 42, 42),
        'Mahogany': (75, 0, 13),
        'Chestnut': (139, 69, 19),
        'Umber': (99, 81, 71),
        'Beige': (245, 245, 220),
        'Tawny': (205, 87, 0),
        'Caramel': (255, 213, 154),
        'Khaki': (240, 230, 140),  
        'Camel': (193, 154, 107),  
    },
}


# Load the color dataset from the CSV file
color_dataset = {}
with open('colours_rgb_shades.csv', mode='r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        color_name, rgb_values = row[0], row[2]
        color_dataset[color_name] = rgb_values

# Create a CSV file to store the results
with open('image_colors.csv', 'w', newline='') as csvfile:
    fieldnames = ['Image Name', 'RGB Color', 'Assigned Color Name', 'Assigned Base Color']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Loop through the image files and process each one
    for image_file in image_files:
        # Load the image
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)

        # Get the image dimensions
        height, width, _ = image.shape

        # Calculate the center coordinates
        center_x, center_y = width // 2, height // 2

        # Define the size of the center region (you can adjust this as needed)
        center_size = 50  # Example size, you can change it

        # Calculate the center region rectangle
        x = center_x - center_size // 2
        y = center_y - center_size // 2
        w = center_size
        h = center_size

        # Crop the center region
        center_region = image[y:y+h, x:x+w]

        # Calculate the main color of the center region (you can use a color recognition library)
        # For simplicity, we will calculate the average color
        main_color = np.mean(center_region, axis=(0, 1))

        # Convert the BGR values to RGB
        main_color_rgb = (int(main_color[2]), int(main_color[1]), int(main_color[0]))

        # Classify the RGB color to the nearest base color
        assigned_base_color = classify_color(main_color_rgb, base_colors)

        # Classify the RGB color to the nearest color name based on the dataset
        assigned_color_name = classify_to_color_name(main_color_rgb, color_dataset)

        print(f"Image Name: {image_file}")
        print(f"RGB Color: {main_color_rgb}")
        print(f"Assigned Color Name: {assigned_color_name}")
        print(f"Assigned Base Color: {assigned_base_color}")

        # Write the results to the CSV file
        writer.writerow({'Image Name': image_file, 'RGB Color': main_color_rgb, 'Assigned Color Name': assigned_color_name, 'Assigned Base Color': assigned_base_color})


Image Name: image_0.jpg
RGB Color: (24, 27, 47)
Assigned Color Name: grey13
Assigned Base Color: Black
Image Name: image_1.jpg
RGB Color: (184, 32, 32)
Assigned Color Name: firebrick
Assigned Base Color: Brown
Image Name: image_10.jpg
RGB Color: (235, 233, 220)
Assigned Color Name: seashell2
Assigned Base Color: White
Image Name: image_11.jpg
RGB Color: (231, 228, 218)
Assigned Color Name: seashell2
Assigned Base Color: White
Image Name: image_12.jpg
RGB Color: (36, 39, 51)
Assigned Color Name: grey16
Assigned Base Color: Black
Image Name: image_13.jpg
RGB Color: (224, 223, 219)
Assigned Color Name: grey87
Assigned Base Color: White
Image Name: image_14.jpg
RGB Color: (192, 204, 221)
Assigned Color Name: azure3
Assigned Base Color: Blue
Image Name: image_15.jpg
RGB Color: (220, 218, 180)
Assigned Color Name: Wheat
Assigned Base Color: White
Image Name: image_16.jpg
RGB Color: (41, 41, 44)
Assigned Color Name: grey16
Assigned Base Color: Black
Image Name: image_17.jpg
RGB Color: (135, 5